In [24]:
#!pip install emoji
#!pip install pydantic==1.10.2.
#!pip3 install spacy
#!pip install gensim
#!pip install wordcloud
#!pip install openpyxl
#!pip install XlsxWriter

In [25]:
#Removing emojis from text
#Refrence 1 : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
#Refrence 2 : https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python

def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.EMOJI_DATA]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub (r'^https?:\/\/.*[\r\n]*', '', text)
    
    #text = re.sub(r'http\S+', '', text)
    return text

In [26]:
from datetime import datetime
import numpy as np

def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)

In [27]:
from pandas import read_excel
data = read_excel("original_tweets.xlsx",sheet_name = 'original tweet data')

In [28]:
import pandas as pd
# Converting the dataset to pandas DataFrame and renaming the columns 
df_original_tweet = pd.DataFrame(data['text'].tolist())

df_original_tweet = df_original_tweet.rename(columns={0:'original_tweets'})
df_original_tweet = df_original_tweet.rename(columns={1:'tweet_id'})
df_original_tweet = df_original_tweet.rename(columns={2:'company_name'})#company_name
df_original_tweet = df_original_tweet.rename(columns={3:'created_at'})#created_at


df_original_tweet['company_name'] = data['company_name']
df_original_tweet['created_at'] = data['created_at']
df_original_tweet['tweet_id'] = data['tweet_id']
df_original_tweet.head(10)

df_original_tweet.shape


(3172, 4)

In [29]:
import emoji
import re
# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
df_original_tweet['emoji_free_tweets'] = df_original_tweet['original_tweets'].apply(call_emoji_free)

#Create a new column with url free tweets

df_original_tweet['sanitized_text'] = df_original_tweet['emoji_free_tweets'].apply(url_free_text)
df_original_tweet.head(5)
df_original_tweet.shape

(3172, 6)

In [30]:
# Filter by company and date
#company_name='Amazon'
#company_name = 'Amazon'
#df_original_tweet = df_original_tweet.loc[df_original_tweet['company_name'] == company_name]
df_original_tweet['created_at'] = pd.to_datetime(df_original_tweet['created_at']) # converting string in date object
df_original_tweet = df_original_tweet.sort_values(by="created_at")

In [31]:
#Remove links and mentions(@)
import re
def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet) 
    tweet = re.sub('http[^\s]+','',tweet)
    return tweet
df_original_tweet['sanitized_text'] = df_original_tweet['sanitized_text'].apply(remove_usernames_links)


In [32]:
#Extract hash tag words into a new column named hash_tag_words
import re
def extract_hashtag_words(tweet):
    hashtags = re.findall("#([a-zA-Z0-9_]{1,50})", tweet)
    return hashtags

df_original_tweet['hash_tag_words'] = df_original_tweet['sanitized_text'].apply(extract_hashtag_words)

In [33]:
# Remove hash tag words from sanitized_text 
import re
def remove_hashtag_words(tweet):
    tweet = re.sub('#([a-zA-Z0-9_]{1,50})','',tweet)
    return tweet

df_original_tweet['sanitized_text'] = df_original_tweet['sanitized_text'].apply(remove_hashtag_words)


In [34]:
#Remove Text(1/2), 1/,2/ etc
import re
def remove_others(tweet):
    tweet = tweet.replace("(", "")
    tweet = tweet.replace(")", "")
    tweet = re.sub("[0-9]/",'',tweet) # remove 1/,2/ etc
    tweet = re.sub("Text [0-9]:",'',tweet) # Text(1/2)
    tweet = re.sub("[0-50].",'',tweet) # 1. 2. etc
    return tweet

df_original_tweet['sanitized_text'] = df_original_tweet['sanitized_text'].apply(remove_others)


In [35]:
df_original_tweet

,original_tweets,company_name,created_at,tweet_id,emoji_free_tweets,sanitized_text,hash_tag_words
0,"Tomorrow, we walk out for real change. #google...",Google,2018-11-01 00:38:58+00:00,1057794087955427328,"Tomorrow, we walk out for real change. #google...","Tomorrow, we walk out for real change.","[googlewalkout, google]"
1,"We, Google employees and contractors, will wal...",Google,2018-11-01 01:19:09+00:00,1057804203895283712,"We, Google employees and contractors, will wal...","We, Google employees and contractors, will wal...",[googlewalkout]
2,Google employees and contractors will be leavi...,Google,2018-11-01 02:46:11+00:00,1057826104965246976,Google employees and contractors will be leavi...,Google employees and contractors will be leavi...,[googlewalkout]
3,@sasha_feather @hypatiadotca Thank you so much...,Google,2018-11-01 03:29:24+00:00,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,Thank you so much for the reminder! We’re on...,[]
4,Text (1/2):\n1. An end to Forced Arbitration i...,Google,2018-11-01 03:30:52+00:00,1057837349290287104,Text (1/2): 1. An end to Forced Arbitration in...,An end to Forced Arbitration in cases of har...,[]
...,...,...,...,...,...,...,...
3168,6/ Maybe the scariest part about all of this i...,Amazon,2021-12-17 20:06:53+00:00,1471934946625429510,6/ Maybe the scariest part about all of this i...,Maybe the scariest part about all of this is ...,[]
3167,"7/ As workers, we’re disturbed. “This report i...",Amazon,2021-12-17 20:06:53+00:00,1471934948336615430,"7/ As workers, we’re disturbed. “This report i...","As workers, we’re disturbed. “This report is ...",[]
3169,8/ Thank you @kavehwaddell @ConsumerReports @g...,Amazon,2021-12-17 20:06:53+00:00,1471934950073151496,8/ Thank you @kavehwaddell @ConsumerReports @g...,Thank you for your important investigative...,[]
3170,1/ I've worked for Apple since August 2015. \n...,Apple,2021-12-20 14:50:08+00:00,1472942398946910211,1/ I've worked for Apple since August 2015. Ha...,I've worked for Apple since August . Harassme...,[appletoo]


In [36]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
document = df_original_tweet['sanitized_text'].values.tolist()
#pprint.pprint(document)
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
#print(spacy.__version__)
nlp = spacy.load('en_core_web_lg')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [37]:
"""
Import Gensim and Wordcloud to use their stopwords as well and use the combined stopwords of ALL as the variable:
ALL_STOP_WORDS
"""

import time
from spacy.tokenizer import Tokenizer
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
# Timing Start
program_start_time = time.time()

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', 'cont', '$', '@','RT','rt','google','amazon','nyc','?','zurich','re','we']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

In [38]:
tokens = []

for doc in tokenizer.pipe(df_original_tweet['sanitized_text'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in ALL_STOP_WORDS:
            doc_tokens.append(token.text.lower()) 
    tokens.append(doc_tokens)

# Makes tokens column
df_original_tweet['unigrams'] = tokens

# Make tokens a string again
#df_original_tweet['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df_original_tweet['unigrams']]

# Timing End
program_end_time = time.time()

# View df
#df[['sanitized_text','tokens','tokens_back_to_text']]
df_original_tweet.shape

(3172, 8)

In [39]:
df_original_tweet.head(10)

,original_tweets,company_name,created_at,tweet_id,emoji_free_tweets,sanitized_text,hash_tag_words,unigrams
0,"Tomorrow, we walk out for real change. #google...",Google,2018-11-01 00:38:58+00:00,1057794087955427328,"Tomorrow, we walk out for real change. #google...","Tomorrow, we walk out for real change.","[googlewalkout, google]","[tomorrow,, walk, real, change., ]"
1,"We, Google employees and contractors, will wal...",Google,2018-11-01 01:19:09+00:00,1057804203895283712,"We, Google employees and contractors, will wal...","We, Google employees and contractors, will wal...",[googlewalkout],"[we,, employees, contractors,, walkout, novemb..."
2,Google employees and contractors will be leavi...,Google,2018-11-01 02:46:11+00:00,1057826104965246976,Google employees and contractors will be leavi...,Google employees and contractors will be leavi...,[googlewalkout],"[employees, contractors, leaving, flyers, desk..."
3,@sasha_feather @hypatiadotca Thank you so much...,Google,2018-11-01 03:29:24+00:00,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,Thank you so much for the reminder! We’re on...,[],"[ , thank, reminder!, we’re, it,, sorry, lapse.]"
4,Text (1/2):\n1. An end to Forced Arbitration i...,Google,2018-11-01 03:30:52+00:00,1057837349290287104,Text (1/2): 1. An end to Forced Arbitration in...,An end to Forced Arbitration in cases of har...,[],"[ , end, forced, arbitration, cases, harassme..."
5,"(2/2):\n4. A clear, uniform, globally inclusiv...",Google,2018-11-01 03:31:55+00:00,1057837611891507200,"(2/2): 4. A clear, uniform, globally inclusive...","A clear, uniform, globally inclusive process...",[],"[ , clear,, uniform,, globally, inclusive, pr..."
6,"@hypatiadotca @sasha_feather That’s awesome, w...",Google,2018-11-01 03:36:14+00:00,1057838702330822656,"@hypatiadotca @sasha_feather That’s awesome, w...","That’s awesome, we’ll use that feature in th...",[],"[ , that’s, awesome,, we’ll, use, feature, fu..."
7,More photos are coming in from the 40+ offices...,Google,2018-11-01 10:46:51+00:00,1057947069506355200,More photos are coming in from the 40+ offices...,More photos are coming in from the + offices w...,[GoogleWalkout],"[photos, coming, +, offices, walking, today!, ..."
8,Another angle of the #GoogleWalkout at Google ...,Google,2018-11-01 10:52:29+00:00,1057948484731584514,Another angle of the #GoogleWalkout at Google ...,Another angle of the at Google Zürich,[GoogleWalkout],"[angle, zürich]"
9,"Google Dublin, we see you! Thanks for coming o...",Google,2018-11-01 11:32:33+00:00,1057958568551464961,"Google Dublin, we see you! Thanks for coming o...","Google Dublin, we see you! Thanks for coming o...",[GoogleWalkout],"[dublin,, you!, thanks, coming, today]"


In [40]:
#Build bigrams and trigrams
import gensim
bigram_phrases = gensim.models.Phrases(df_original_tweet['unigrams'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_phrases = gensim.models.Phrases(bigram_phrases[df_original_tweet['unigrams']], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram[bigram[doc]] for doc in texts]

bigrams = make_bigrams(df_original_tweet['unigrams'])
trigrams = make_trigrams(df_original_tweet['unigrams'])
#df_original_tweet['bigrams'] = bigrams
df_original_tweet['n_grams'] = trigrams #Includes bigrams trigrams and unigrams


In [41]:
#Filter out unigrams
import pprint
import re

def filter_unigrams(df):
    bi_tri_list = []
    uni_list = []
    data = df
    for index, row in data.iterrows():
        bi_tri = []
        uni = []
        if(type(row['n_grams'])!=None):
            for word in row['n_grams']:
                #if word.find('_')!=-1:
                if re.search("([a-zA-Z0-9])_([a-zA-Z0-9])", word):
                    #pprint.pprint(word)
                    bi_tri.append(word)
                else:
                    uni.append(word)

            bi_tri_list.append(bi_tri)
            uni_list.append(uni)
            
    return bi_tri_list,uni_list 


bi_tri_list, uni_list = filter_unigrams(df_original_tweet)
 
#pprint.pprint(bi_tri_list)
#pprint.pprint(uni_list)
df_original_tweet['unigrams'] = uni_list
df_original_tweet['bigrams_trigrams'] = bi_tri_list


In [42]:
#print(data_words_bigrams)

#These are names of places or people.
words_to_remove_set = {"amznclimate_fridaysforfuture",
                         "copley_square","googlewac_msworkers",
                         "emahlee_marencosta",
                     "jeff_bezos","meredith_clairewaves","timnit_gebru","mountain_view",
                         "san_bernardino",
                     "dr_timnit","dr_gebru","dr_gebrus",
                         "san_francisco","new_york",
                         "bay_area","timnitgebru",
                         "meredith","wayfairwalkout", "tweetsofsandra_week","riot_games", "githubdropice_notechforice","notechforice_githubdropice" }
                        # {"near_warehouses","stay_tuned",
                        # "board_directors","temps_vendors","software_engineer",
                        # "press_release","shareholder_meeting","gas_companies","tweetsofsandra_week"}

            
def filter_places_and_people_names(df_original_tweet):
    data = df_original_tweet
    bigrams_filtered = []
    unigrams_filtered = []
    ngrams_filtered = []
    for index, row in data.iterrows():
        tmp = []
        for word in row['bigrams_trigrams']:    
            if word not in words_to_remove_set:
                tmp.append(word)
        bigrams_filtered.append(tmp)
        tmp = []
        for word in row['unigrams']:    
            if word not in words_to_remove_set:
                tmp.append(word)
        unigrams_filtered.append(tmp)
        tmp = []
        for word in row['n_grams']:    
            if word not in words_to_remove_set:
                tmp.append(word)
        ngrams_filtered.append(tmp)
        
    return unigrams_filtered, bigrams_filtered, ngrams_filtered
            
        
unigrams_filtered, bigrams_filtered, ngrams_filtered =  filter_places_and_people_names(df_original_tweet)
df_original_tweet['unigrams_filtered'] = unigrams_filtered
df_original_tweet['bigrams_filtered'] = bigrams_filtered
df_original_tweet['ngrams_filtered'] = ngrams_filtered



In [43]:
# Make tokens a string again
df_original_tweet['unigrams_to_text'] = [' '.join(map(str, l)) for l in df_original_tweet['unigrams_filtered']]
df_original_tweet['bigrams_to_text'] = [' '.join(map(str, l)) for l in df_original_tweet['bigrams_filtered']]
df_original_tweet['ngram_to_text'] = [' '.join(map(str, l)) for l in df_original_tweet['ngrams_filtered']]


In [44]:
df_santanize = df_original_tweet[['tweet_id','original_tweets','company_name','ngram_to_text','unigrams_to_text','bigrams_to_text']].copy()
df_santanize.shape

(3172, 6)

In [46]:
df_santanize.head(100)

,tweet_id,original_tweets,company_name,ngram_to_text,unigrams_to_text,bigrams_to_text
0,1057794087955427328,"Tomorrow, we walk out for real change. #google...",Google,"tomorrow, walk real change.","tomorrow, walk real change.",
1,1057804203895283712,"We, Google employees and contractors, will wal...",Google,"we, employees contractors, walkout november :a...","we, employees contractors, walkout november :a...",
2,1057826104965246976,Google employees and contractors will be leavi...,Google,employees contractors leaving flyers desks tom...,employees contractors leaving flyers desks tom...,
3,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,Google,"thank reminder! we’re it, sorry lapse.","thank reminder! we’re it, sorry lapse.",
4,1057837349290287104,Text (1/2):\n1. An end to Forced Arbitration i...,Google,end forced_arbitration cases harassment dis...,end cases harassment discrimination current...,forced_arbitration sexual_harassment
...,...,...,...,...,...,...
95,1058058822500188160,Hundreds of Google employees (some carrying si...,Google,hundreds employees carrying signs gathering ce...,hundreds employees carrying signs gathering ce...,sexual_misconduct
96,1058059546332139520,greatest sign or greatest sign #googlewalkout ...,Google,greatest sign greatest sign,greatest sign greatest sign,
97,1058059737932140545,"We see you, Google Kirkland! #GoogleWalkout ht...",Google,"you, kirkland!","you, kirkland!",
98,1058059938344456193,"Thanks for braving the cold, Google Seattle! #...",Google,"thanks braving cold, seattle!","thanks braving cold, seattle!",


In [47]:

df_santanize['tweet_id'] = df_santanize['tweet_id'].apply(str)

# create excel writer object
writer1 = pd.ExcelWriter('sanitized_data.xlsx')
# write dataframe to excel
df_santanize.to_excel(writer1, sheet_name = 'sanitized_text', engine="xlsxwriter")

# save the excel
writer1.save()
#writer.close()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.
